In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
model = YOLO('models/yolov8x-pose.pt') 

In [3]:
img = cv2.imread('examples/images/RDR2 2022-01-16 19-54-04.png')

In [15]:
image_path = 'examples/videos/V12.mp4'
result = model.track(image_path, device='0', show=True, verbose=False, stream=True)
cv2.destroyAllWindows()

In [5]:
def _extract_keypoints(self, data):
    d = {}
    max_persons = 10

    for i, frame in enumerate(data):
        if i == max_persons:
            self.keypoints_batch.append(list(d.values()))
            d = {}
            break  # Break the loop if more than 10 frames are processed

        for i in frame:
            keypoints = i["keypoints"]
            x = np.array(keypoints["x"]) / self.orig_shape[1]
            y = np.array(keypoints["y"]) / self.orig_shape[0]
            XY = [j for j in zip(x, y)]

            XY = np.array(XY).reshape(34).tolist()

            track_id_keypoints = d.get(i["track_id"], [])
            track_id_keypoints.append(XY)
            d[i["track_id"]] = track_id_keypoints

    # Fill the rest of the frames with zeros if fewer than 10
    while len(d) < max_persons:
        d[len(d)] = np.zeros((17, 2)).tolist()

    # Convert the batch list into np.array() with uniform shape
    self.keypoints_batch.append(list(d.values()))
    self.keypoints_batch = np.array(self.keypoints_batch)

    # Ensure uniform shape by padding or truncating
    self.keypoints_batch = np.array([np.pad(item, ((0, max_persons - len(item)), (0, 0)), mode='constant')[:max_persons] for item in self.keypoints_batch])

# Archive 1

In [7]:
def _convert_result_to_string(result):
    output = '['
    for i, r in enumerate(result):
        if i==20: break
        output += r.tojson() + ','
    output = output[:-1]+']'
    cv2.destroyAllWindows()
    return output, r.orig_shape

In [16]:
output, orig_shape = _convert_result_to_string(result)
data = eval(output)

d={}
for i in data[0]:
  keypoints = i["keypoints"]
  x = np.array(keypoints["x"])/orig_shape[1]
  y = np.array(keypoints["y"])/orig_shape[0]
  
  XY = [j for j in zip(x, y) ]

  track_id_keypints = (d.get(i["track_id"],[]))
  track_id_keypints.append(XY)
  d[i["track_id"]] = track_id_keypints

d

<class 'numpy.ndarray'>
(17, 2)
<class 'numpy.ndarray'>
(17, 2)


{1: array([], dtype=float64), 2: array([], dtype=float64)}

In [30]:
for k in d.keys():
    for x,y in d[k][0]:
        cv2.circle(img, (int(x),int(y)), 1, (0,0,255), -1)

In [34]:
cv2.imshow("s",img)
cv2.waitKey(2000)
cv2.destroyAllWindows()

In [8]:
video_path = 'examples/videos/V53.mp4'
result = model.track(video_path, show=True, stream=True, save=True, device='0', persist=True)
cv2.destroyAllWindows()

In [9]:
for r in result:
    print(r.tojson)
cv2.destroyAllWindows()


<bound method Results.tojson of ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: ultralytics.engine.results.Keypoints object
masks: None
names: {0: 'person'}
obb: None
orig_img: array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
     

In [11]:
cv2.destroyAllWindows()


In [5]:
print(len(result))
result[0].keypoints.xyn.shape

68


torch.Size([1, 17, 2])

In [13]:
import cv2
import time

# Create a video capture object
cap = cv2.VideoCapture(0)  # Use 0 for webcam, or provide a video file path

cap.set(cv2.CAP_PROP_FPS, 30)

# Initialize variables
frames = []
frame_count = 0

ret, frame = cap.read()
begin_time = time.time()

start_time = time.time()

results = []

for i in range(10):
    ret, frame = cap.read()
    frames.append(frame)
    

while True:
    # Capture a frame
    ret, frame = cap.read()

    # If frame is read correctly, append it to the frames list
    if ret:
        frames.append(frame)
        frame_count += 1

        # If 5 frames have been captured or a second has passed, process them
        if time.time() - start_time >= 1:
            print("frame count", frame_count, end='\t')
            
            pridict_start = time.time()
            result = model.track(frame, show=True, save=False, imgsz=640, device='0', verbose=False)
            print(time.time()-pridict_start)
            
            keypoints = []
            for frame_result in result:
                keypoints.append(frame_result.keypoints.xyn)
            results.append(keypoints)


            # Reset the frames list and frame count
            frames = []
            frame_count = 0
            start_time = time.time()
            cv2.imshow("Frame", frame)
            if cv2.waitKey(1) == ord('q'):
                break
            continue

    cv2.imshow("Frame", frame)
    if cv2.waitKey(100) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
print(time.time() - begin_time, )


frame count 3	0.0646524429321289
frame count 7	0.14597296714782715
frame count 10	0.08529496192932129
frame count 10	0.16553187370300293
frame count 10	0.0847012996673584
frame count 10	0.17261672019958496
7.884396076202393


In [11]:
cap.release()
cv2.destroyAllWindows()

In [64]:
print(len(results))
# for i in results:
#     print(len(i))

2


In [7]:
import cv2
import time

# Create a video capture object
cap = cv2.VideoCapture("examples/videos/V12.MOV")  # Use 0 for webcam, or provide a video file path

cap.set(cv2.CAP_PROP_FPS, 30)

# Initialize variables
frames = []
frame_count = 0
buffer_size = 5
results = []

for i in range(buffer_size):
    ret, frame = cap.read()
    frames.append(frame)
    
try:
    while True:

        # Capture a frame
        ret, frame = cap.read()

        # If frame is read correctly, append it to the frames list
        if ret:
            frames.append(frame)

            frames.pop(0)

            frame_count += 1
            # print("frame count", frame_count, end='\t')
            
            pridict_start = time.time()
            result = model.track(frame, show=True, imgsz=640, device='0', verbose=False, stream=False)

            # print(time.time()-pridict_start)

        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

except KeyboardInterrupt:
    cap.release()
    cv2.destroyAllWindows()
    # print(time.time() - begin_time, )


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


In [11]:
cap.release()
cv2.destroyAllWindows()

In [110]:
len(list(frames))

5

In [12]:
len(result)

5

In [15]:
print(result.tojson())

[
  {
    "name": "person",
    "class": 0,
    "confidence": 0.9318608641624451,
    "box": {
      "x1": 483.0,
      "y1": 186.0,
      "x2": 651.0,
      "y2": 716.0
    },
    "track_id": 1,
    "keypoints": {
      "x": [
        536.0521240234375,
        551.146484375,
        532.093017578125,
        581.8046875,
        0.0,
        611.0513916015625,
        511.95953369140625,
        611.4508056640625,
        508.6263427734375,
        530.4464111328125,
        506.5220031738281,
        608.506591796875,
        545.0972900390625,
        596.7869262695312,
        569.7277221679688,
        0.0,
        0.0
      ],
      "y": [
        248.3331298828125,
        239.09902954101562,
        236.3554229736328,
        248.61923217773438,
        0.0,
        309.15191650390625,
        310.73486328125,
        409.00543212890625,
        404.8263244628906,
        422.604736328125,
        483.02374267578125,
        462.35333251953125,
        462.51031494140625,
    

# Archive 2